In [ ]:
import pandas as pd
df = pd.read_csv('main.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54397 entries, 0 to 54396
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Week                             54397 non-null  int64  
 1   Agent_Tenure                     54397 non-null  int64  
 2   Customer_Tenure                  54397 non-null  int64  
 3   category_1                       54397 non-null  int64  
 4   category_2                       54397 non-null  int64  
 5   category_3                       54397 non-null  int64  
 6   category_4                       54397 non-null  int64  
 7   est_spend                        54397 non-null  int64  
 8   uniqueid                         54397 non-null  int64  
 9   P_NPS_Detract_Pass_ProPROMOTERS  54397 non-null  float64
 10  Weight                           54397 non-null  float64
dtypes: float64(2), int64(9)
memory usage: 4.6 MB


In [ ]:
import numpy as np

In [ ]:
df['est_spend'].astype('float64')

0           7.0
1         192.0
2           6.0
3         428.0
4        6559.0
          ...  
54392     165.0
54393      25.0
54394     947.0
54395      11.0
54396       8.0
Name: est_spend, Length: 54397, dtype: float64

In [ ]:
sum_cat1 = df.groupby('Week').apply(lambda x: np.sum(x.category_1*x.Weight))
sum_cat2 = df.groupby('Week').apply(lambda x: np.sum(x.category_2*x.Weight))
sum_cat3 = df.groupby('Week').apply(lambda x: np.sum(x.category_3*x.Weight))
sum_cat4 = df.groupby('Week').apply(lambda x: np.sum(x.category_4*x.Weight))
agent_tenure = df.groupby('Week').apply(lambda x: np.average(x.Agent_Tenure, weights = x.Weight))
cust_tenure = df.groupby('Week').apply(lambda x: np.average(x.Customer_Tenure, weights = x.Weight))
est_spend = df.groupby('Week').apply(lambda x: np.sum(x.est_spend*x.Weight))
probability = df.groupby('Week').apply(lambda x: np.average(x.P_NPS_Detract_Pass_ProPROMOTERS, weights = x.Weight))

In [ ]:
df1 = pd.concat([sum_cat1, sum_cat2, sum_cat3, sum_cat4, agent_tenure, cust_tenure, est_spend, probability], axis = 1)

In [ ]:
df1 = df1.rename(columns = {0:'cat1',1:'cat2',2:'cat3',3:'cat4',4:'agent_tenure',5:'cust_tenure',6:'est_spend',7:'probability'})

In [ ]:
import statsmodels.api as sm
X = sm.add_constant(df1[['cat1', 'cat2','cat3','cat4','agent_tenure','cust_tenure','est_spend']])
Y = df1['probability']
model = sm.OLS(Y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            probability   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Fri, 03 Mar 2023   Prob (F-statistic):                nan
Time:                        22:56:08   Log-Likelihood:                 208.02
No. Observations:                   7   AIC:                            -402.0
Df Residuals:                       0   BIC:                            -402.4
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         3.069e-06        inf          0   

/usr/local/lib/python3.8/dist-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 7 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
/usr/local/lib/python3.8/dist-packages/statsmodels/regression/linear_model.py:1765: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/usr/local/lib/python3.8/dist-packages/statsmodels/regression/linear_model.py:1765: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/usr/local/lib/python3.8/dist-packages/statsmodels/regression/linear_model.py:1687: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


In [ ]:
mean_prob = df1['probability'].mean()

In [ ]:
mean_prob

0.6192275062718249

In [ ]:
round(((model.params['cat2']*0.1)/mean_prob)*100,2)

-0.01

In [ ]:
round(((model.params['cat3']*0.1)/mean_prob)*100,2)

0.01